# Testing the function of Tensorboard

## Importing libraries

In [13]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

## Loading dataset

In [2]:
housing = fetch_california_housing()

In [3]:
df_data = pd.DataFrame(housing.data, columns = housing.feature_names)

In [4]:
df_data['target'] = pd.Series(housing.target)

In [5]:
df_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Splitting the data

- We will be splitting the data into train, validation and test sets.

In [6]:
x = np.array(df_data.drop(columns = 'target', axis = 1))
y = np.array(df_data.target)

In [7]:
x_train_full, x_test, y_train_full, y_test = train_test_split(x, y, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state = 42)

## Scaling the data

- Scaling the data using Standard Scaler

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

## Building the model

- The NN model will be having 2 hidden layers with 30 units each.

In [9]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(30, activation = 'relu', input_shape = [8]))
model.add(keras.layers.Dense(30, activation = 'relu'))
model.add(keras.layers.Dense(1))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 1,231
Trainable params: 1,231
Non-trainable params: 0
_________________________________________________________________


## Compiling the model

- Loss function is mean squared error.
- Optimizer is SGD with a learning rate of 1e-3.

In [11]:
model.compile(loss = 'mse', optimizer = keras.optimizers.SGD(lr = 1e-3))

## Defining root log directory

In [12]:
root_log_dir = os.path.join(os.curdir, 'logs')

In [14]:
def current_log_dir():
    run_id = time.strftime('run_%Y_%m_%d_%H_%M_%S')
    return os.path.join(root_log_dir, run_id)

In [15]:
run_log_dir = current_log_dir()

In [16]:
run_log_dir

'./logs/run_2020_09_01_13_26_43'

- We have defined the root log directory to save our logs.
- Then we defined a function which updates the log calculated in different file, with the file names having the time details of the log.
- This is then added in the root log directory.

## Tensorboard callback

In [17]:
tensorboard_cb = keras.callbacks.TensorBoard(run_log_dir)

- The callback functoin will take care of creating the log directories and during training it will create event files and write summaries to them.

## Training the model

In [18]:
history = model.fit(x_train, y_train, epochs = 30, validation_data = (x_valid, y_valid), callbacks = [tensorboard_cb])

Epoch 1/30
  1/363 [..............................] - ETA: 0s - loss: 3.9116WARNING:tensorflow:From /home/gokul/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
363/363 [==============================] - 1s 1ms/step - loss: 1.4961 - val_loss: 1.3095
Epoch 2/30
363/363 [==============================] - 0s 890us/step - loss: 0.7685 - val_loss: 0.7242
Epoch 3/30
363/363 [==============================] - 0s 888us/step - loss: 0.6604 - val_loss: 0.6066
Epoch 4/30
363/363 [==============================] - 0s 892us/step - loss: 0.6002 - val_loss: 0.5627
Epoch 5/30
363/363 [==============================] - 0s 886us/step - loss: 0.5568 - val_loss: 0.5168
Epoch 6/30
363/363 [==============================] - 0s 845us/step - loss: 0.5242 - val_loss: 0.4864
Epoch 7/30
363/363 [===================